## Домашнее задание 4.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# data = pd.read_csv('../raw_data/transaction_data.csv')
data = pd.read_csv('D:/WORK/BI/GeekBrains/Course_Recommender_systems/Data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
# item_features = pd.read_csv('../raw_data/product.csv')
item_features = pd.read_csv('D:/WORK/BI/GeekBrains/Course_Recommender_systems/Data/product.csv')

item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


#### Функция prefilter_items (ограничения реализованы)

In [6]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 4027


In [7]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,818980,819063,819227,819304,819308,819330,819518,819594,819643,819840,...,15596515,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [9]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [10]:
%%time

model = AlternatingLeastSquares(factors = 20, 
                                regularization = 0.001,
                                iterations = 15, 
                                calculate_training_loss = True, 
                                num_threads = 4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 3.16 s


-----

# Домашнее задание

1. Изучите структуру модуля src
2. Перенесите функции prefilter_items и postfilter_items в модуль src.utils.py
3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.utils.py.  
postfilter можно пока оставить пустой (или как-то бороться с товаром 999999) - на Ваше усмотрение.  
В prefilter нужно реализовать закомментированные ограничения.
4. Создайте модуль src.recommenders.py. Напишите код для класса ниже и положите его в src.recommenders.py  
get_similar_items_recommendation и get_similar_users_recommendation - реализуйте их в качетсве методов класса.  
Посчитайте с помощью них рекомендации и найдите precision@5
5. Проверьте, что все модули корректно импортируются
6. Если вы еще не прочитали [статью](https://habr.com/ru/company/hh/blog/347276/) о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте

**Внимание!** модуль для вашего удобства - дополняйте его теми функциями, которые Вам пригодятся.  
Формальное задание только на prefiletr_items 

#### Проверка, что все работает (модули и функции)

In [11]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items, get_similar_items_recommendation, get_similar_users_recommendation
from src.recommenders import MainRecommender

#### Задание 3.1 - Проверка функции get_similar_items_recommendation (модуль utils)

In [12]:
print(get_similar_items_recommendation(2500, model, data_train, itemid_to_id, id_to_itemid))
print(get_similar_items_recommendation(2375, model, data_train, itemid_to_id, id_to_itemid))
print(get_similar_items_recommendation(3,    model, data_train, itemid_to_id, id_to_itemid))

[12731808, 1083740, 825618, 6533362, 869604]
[876695, 1133312, 1077373, 919681, 819308]
[901543, 1110292, 864615, 821562, 9189814]


#### Задание 3.2 - Проверка функции get_similar_users_recommendation (модуль utils)

In [13]:
print(get_similar_users_recommendation(2500, model, user_item_matrix, userid_to_id, itemid_to_id, id_to_itemid ))
print(get_similar_users_recommendation(2375, model, user_item_matrix, userid_to_id, itemid_to_id, id_to_itemid ))
print(get_similar_users_recommendation(3,    model, user_item_matrix, userid_to_id, itemid_to_id, id_to_itemid ))

[883186, 891505, 967476, 1044259, 972657, 925282]
[1049735, 1057749, 1011926, 1031864, 900770, 1019479]
[1048257, 1018588, 850944, 938187, 966891, 990440]


#### Задание 3.3 - В функции prefilter_items все ограничения реализованы (см модуль utils + фрагмент когда выше, как тест)

In [14]:
mr = MainRecommender(data_train)

#### Задание 4. модуль recommender.py

#### функция get_similar_items_recommendation

In [15]:
print(mr.get_similar_items_recommendation(2500))
print(mr.get_similar_items_recommendation(2375))
print(mr.get_similar_items_recommendation(3))

[9879410, 7410343, 825618, 6533608, 950384]
[987557, 1133312, 967205, 919681, 1037337]
[845774, 1124694, 863506, 821562, 13842214]


#### функция get_similar_users_recommendation

In [17]:
print(mr.get_similar_users_recommendation(2500))
print(mr.get_similar_users_recommendation(2375))
print(mr.get_similar_users_recommendation(3))

[883186, 866540, 12384365, 980823, 1077143, 12518330]
[1047769, 932298, 957232, 1070272, 1054945, 971983]
[919999, 10284946, 1103556, 936355, 849315, 879635]


#### Тест

In [18]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns = ['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


#### рекомендации и Precision

In [22]:
%%time
    
result['similar_items'] = result['user_id'].apply(lambda x: mr.get_similar_items_recommendation(x, N = 5))

Wall time: 30.2 s


In [23]:
result.apply(lambda row: precision_at_k(row['similar_items'], row['actual']), axis = 1).mean()

0.04183070866141756